# Analyzing Hubway Trips in SQL

In [38]:
#You will have to install ipython-sql using the following command:

!pip install ipython-sql --upgrade

#Just Execute the cell and it will install

Defaulting to user installation because normal site-packages is not writeable


In [39]:
# If your file is in another Directory (Without Space in Name of Directory) : %sql sqlite:///Hello/Another_Folder/hubway_small_ok.db

# If your file is in Same Directory : %sql sqlite:///hubway_small_ok.db

%reload_ext sql
%sql sqlite:///hubway.db
%config SqlMagic.autocommit=False

**Please write your queries below in the empty boxes same as you write in a SQL Window**

Just append **%sql** before your query like this '%sql Select * from Trips limit 5;'

In [40]:
%%sql

Select * from Trips limit 2

 * sqlite:///hubway.db
Done.


id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
1,9,2011-07-28 10:12:00,23,2011-07-28 10:12:00,23,B00468,Registered,'97217,1976.0,Male
2,220,2011-07-28 10:21:00,23,2011-07-28 10:25:00,23,B00554,Registered,'02215,1966.0,Male


**About the Dataset and Schema**

The database has two tables, trips and stations. To begin with, we'll look at the trips table. It contains the following columns:


**Trips:**

id — A unique integer that serves as a reference for each trip

duration — The duration of the trip, measured in seconds

start_date — The date and time the trip began

start_station — An integer that corresponds to the id column in the stations table for the station the trip started at

end_date — The date and time the trip ended

end_station — The 'id' of the station the trip ended at

bike_number — Hubway's unique identifier for the bike used on the trip

sub_type — The subscription type of the user. "Registered" for users with a membership, "Casual" for users without a membership

zip_code — The zip code of the user (only available for registered members)

birth_date — The birth year of the user (only available for registered members)

gender — The gender of the user (only available for registered members)


**Station:**


id — A unique identifier for each station (corresponds to the start_station and end_station columns in the trips table)

station — The station name

municipality — The municipality that the station is in (Boston, Brookline, Cambridge or Somerville)

lat — The latitude of the station

lng — The longitude of the station

## Analytical Questions Test in SQL

Q1. What was the duration of the longest trip? What was the average trip duration? What was the smallest trip duration?

In [41]:

%%sql

Select min(duration) smallest_trip ,max(duration) longest_trip ,avg(duration) Average_trip  from trips

 * sqlite:///hubway.db
Done.


smallest_trip,longest_trip,Average_trip
0,9999,912.4096819046612


Q2. How many trips were taken by 'Registered' users?

Hint: Use Sub Type to Filter by Where

In [42]:
%%sql

SELECT COUNT(*) AS registered_trips_count
FROM trips
WHERE sub_type = 'Registered';

 * sqlite:///hubway.db
Done.


registered_trips_count
1105192


Q3. How many trips were taken by male users in comparison to Female users? for Registered users only

Hint: Group By Gender and Calculate Count

In [43]:
%%sql

SELECT gender, COUNT(*) AS trip_count
FROM trips
WHERE sub_type = 'Registered'
GROUP BY gender;

 * sqlite:///hubway.db
Done.


gender,trip_count
Female,271333
Female,1
Male,833858


Q4. Do registered or casual users take longer trips?

Hint: Group By Sub Type and Calculate Average

In [44]:
%%sql
SELECT sub_type, AVG(duration) AS avg_trip_duration
FROM trips
GROUP BY sub_type;

 * sqlite:///hubway.db
Done.


sub_type,avg_trip_duration
Casual,1519.6438967403815
Registered,657.0260669639302


Q5. Which bike was used for the most trips?

Hint : Use Bike Number and Limit Results to 1

In [45]:
%%sql
SELECT bike_number, COUNT(*) AS trip_count
FROM trips
GROUP BY bike_number
ORDER BY trip_count DESC
LIMIT 1;

 * sqlite:///hubway.db
Done.


bike_number,trip_count
B00490,2120


Q6. What is the average duration of trips by users over the age of 30?

Hint : DoB is already in years and use Airthmetic operator to caluculate Age like (2021-DOB)

Link : #GETTING AGE IN SQLLITE : https://stackoverflow.com/questions/49929953/how-to-calculate-approximate-age-based-on-birth-year

In [46]:
%%sql
SELECT AVG(duration) AS avg_duration_over_30
FROM trips
WHERE strftime('%Y', 'now') - birth_date > 30;

 * sqlite:///hubway.db
Done.


avg_duration_over_30
657.7647710963483


Q7. Which stations are most frequently used for round trips?

Hint: Round trip is considered as where Start and Stop Stations are same. Limit to top 5 Stations

In [47]:
%%sql
SELECT start_station AS station_id, COUNT(*) AS round_trip_count
FROM trips
WHERE start_station = end_station
GROUP BY start_station
ORDER BY round_trip_count DESC
LIMIT 5;

 * sqlite:///hubway.db
Done.


station_id,round_trip_count
58,3064
36,2548
42,2163
53,2144
52,1636


Q8. How many trips start and end in different municipalities?

Hint: We need to JOIN the trips table to the stations table twice. Once ON the start_station column and then ON the end_station

In [48]:
%%sql
SELECT COUNT(*) AS trips_different_municipalities
FROM trips t
JOIN stations s_start ON t.start_station = s_start.id
JOIN stations s_end ON t.end_station = s_end.id
WHERE s_start.municipality <> s_end.municipality;

 * sqlite:///hubway.db
Done.


trips_different_municipalities
309748


Q9. How many trips incurred additional fees (lasted longer than 30 minutes)?

Hint: Use an Arithmetic function to calculate Duration in Minutes and Use in Where Clause

In [86]:
%%sql
SELECT COUNT(*) AS trips_with_additional_fees
FROM trips
WHERE duration > 30 * 60;

 * sqlite:///hubway.db
Done.


trips_with_additional_fees
123155


Q10. Which bike was used for the longest total time? Provide the answer in Hours

Hint: Use an Group by Bike Number to sum Duration in Seconds and Limit Result to 1

In [50]:
%%sql

SELECT bike_number, SUM(duration) AS total_duration_seconds
FROM trips
GROUP BY bike_number
ORDER BY total_duration_seconds DESC
LIMIT 1;

 * sqlite:///hubway.db
Done.


bike_number,total_duration_seconds
B00490,2058643


Q11. Did registered or casual users take more round trips?

Hint: Recall Q7 and Modify your query to answer by Sub Type

In [51]:
%%sql
SELECT sub_type, COUNT(*) AS round_trips_count
FROM trips
WHERE start_station = end_station
GROUP BY sub_type;

 * sqlite:///hubway.db
Done.


sub_type,round_trips_count
Casual,41427
Registered,31635


Q12. Which municipality had the most frequent Station End?

Hint: Join the Station table on End Station and then Group by Municipality to find answer

In [52]:
%%sql

SELECT s.municipality, COUNT(*) AS station_end_count
FROM trips t
JOIN stations s ON t.end_station = s.id
GROUP BY s.municipality
ORDER BY station_end_count DESC
LIMIT 1;

 * sqlite:///hubway.db
Done.


municipality,station_end_count
Boston,1212364


Q13. Which From and To Routes are Frequent with Regards to Municipalities?

Hint: Get Start and End Municipalities and Group by Both Attributes for Count of Rides

In [53]:
%%sql

SELECT 
    s_start.municipality AS from_municipality,
    s_end.municipality AS to_municipality,
    COUNT(*) AS route_count
FROM trips t
JOIN stations s_start ON t.start_station = s_start.id
JOIN stations s_end ON t.end_station = s_end.id
WHERE s_start.municipality <> s_end.municipality
GROUP BY s_start.municipality, s_end.municipality
ORDER BY route_count DESC;

 * sqlite:///hubway.db
Done.


from_municipality,to_municipality,route_count
Boston,Cambridge,110968
Cambridge,Boston,110078
Cambridge,Somerville,20998
Somerville,Cambridge,20765
Boston,Brookline,14838
Brookline,Boston,14573
Boston,Somerville,6071
Somerville,Boston,5903
Brookline,Cambridge,2738
Cambridge,Brookline,2566


Q14. Which Age Band in "Registered Users" has the most Rides? Calculate Age Bands based on interval of 10 and Cut Off by 70 or Above

Hint: Use Case When to Create Range of Value Bands as per Above Criteria then Group By. Use DoB Years to Calculate Age First (https://youtu.be/G46GIIRbO-g?t=76)

In [54]:
%%sql

SELECT 
    CASE
        WHEN (2024 - birth_date) <= 10 THEN '1-10'
        WHEN (2024 - birth_date) <= 20 THEN '11-20'
        WHEN (2024 - birth_date) <= 30 THEN '21-30'
        WHEN (2024 - birth_date) <= 40 THEN '31-40'
        WHEN (2024 - birth_date) <= 50 THEN '41-50'
        WHEN (2024 - birth_date) <= 60 THEN '51-60'
        WHEN (2024 - birth_date) <= 70 THEN '61-70'
        ELSE '70+'
    END AS age_band,
    COUNT(*) AS ride_count
FROM trips
WHERE sub_type = 'Registered'
GROUP BY age_band
ORDER BY ride_count DESC
LIMIT 1;


 * sqlite:///hubway.db
Done.


age_band,ride_count
70+,768996


# Analyzing Hubway Trips in Python

In [55]:
# MANDATORY CELL TO EXECUTE FIRST SO WE HAVE DATA IN THE RIGHT TYPE

import sqlite3

import pandas as pd, numpy as np

cnx = sqlite3.connect('hubway.db')

cnx.cursor().executescript("""

UPDATE trips SET duration       = NULL WHERE duration IS '';
UPDATE trips SET start_date     = NULL WHERE start_date IS '';
UPDATE trips SET start_station  = NULL WHERE start_station IS '';
UPDATE trips SET end_date       = NULL WHERE end_date IS '';
UPDATE trips SET end_station    = NULL WHERE end_station IS '';
UPDATE trips SET sub_type       = NULL WHERE sub_type IS '';
UPDATE trips SET zip_code       = NULL WHERE zip_code IS '';
UPDATE trips SET birth_date     = NULL WHERE birth_date IS '';
UPDATE trips SET gender         = NULL WHERE gender IS '';

""")

trips=pd.read_sql_query("SELECT * FROM trips", cnx)
stations=pd.read_sql_query("SELECT * FROM stations", cnx)

print(len(trips))
print(len(stations))

trips=trips\
    .replace(r'^\s*$', np.nan, regex=True)\
    .astype({\
            'start_date':'datetime64[ns]'\
            ,'end_date':'datetime64[ns]'\
            ,'start_station':pd.Int64Dtype()\
            ,'end_station':pd.Int64Dtype()\
            ,'birth_date':pd.Int64Dtype()\
            })

stations=stations\
    .replace(r'^\s*$', np.nan, regex=True)\
    .astype({\
            'lat':'float64'\
            ,'lng':'float64'\
            })

print(len(trips))
print(len(stations))

cnx.close()

1570001
142
1570001
142


In [57]:
# CHECKING IF THE DATA IS LOAD INTO RIGHT TABLES IN PYTHON DF

print(trips.head(5))

print('\n ------- \n')

print(stations.head(5))

   id  duration          start_date  start_station            end_date  \
0   1         9 2011-07-28 10:12:00             23 2011-07-28 10:12:00   
1   2       220 2011-07-28 10:21:00             23 2011-07-28 10:25:00   
2   3        56 2011-07-28 10:33:00             23 2011-07-28 10:34:00   
3   4        64 2011-07-28 10:35:00             23 2011-07-28 10:36:00   
4   5        12 2011-07-28 10:37:00             23 2011-07-28 10:37:00   

   end_station bike_number    sub_type zip_code  birth_date  gender  
0           23      B00468  Registered   '97217        1976    Male  
1           23      B00554  Registered   '02215        1966    Male  
2           23      B00456  Registered   '02108        1943    Male  
3           23      B00554  Registered   '02116        1981  Female  
4           23      B00554  Registered   '97214        1983  Female  

 ------- 

   id                             station municipality        lat        lng
0   3              Colleges of the Fenway     

In [58]:
# EXAMPLE SOLUTION, REMEMBER YOUR ANSWER SHOULD BE PRECISE AND ON POINT (SORTED AND IN RIGHT FORMAT)

##PYTHON SOLUTION

trips.agg({'duration': ['mean', 'min', 'max']}).T

,mean,min,max
duration,912.409682,0.0,9999.0


In [97]:
registered_trips_count = trips[trips['sub_type'] == 'Registered'].shape[0]
registered_trips_count

1105192

In [98]:
trip_count_by_gender = trips[(trips['sub_type'] == 'Registered') & (trips['gender'].notnull())]['gender'].value_counts()
trip_count_by_gender

gender
Male        833858
Female      271333
Female\n         1
Name: count, dtype: int64

In [99]:
avg_trip_duration_by_sub_type = trips.groupby('sub_type')['duration'].mean()

avg_trip_duration_by_sub_type

sub_type
Casual        1519.643897
Registered     657.026067
Name: duration, dtype: float64

In [62]:
bike_counts = trips['bike_number'].value_counts()
most_used_bike = bike_counts.idxmax()
trip_count_most_used_bike = bike_counts.max()
print("Bike number used for the most trips:", most_used_bike)
print("Number of trips for the most used bike:", trip_count_most_used_bike)


Bike number used for the most trips: B00490
Number of trips for the most used bike: 2120


In [100]:
from datetime import datetime

trips['birth_date'] = pd.to_datetime(trips['birth_date'])
trips['age'] = datetime.now().year - trips['birth_date'].dt.year
avg_duration_over_30 = trips[trips['age'] > 30]['duration'].mean()
print("Average duration of trips by users over the age of 30:", avg_duration_over_30)


Average duration of trips by users over the age of 30: 657.8409386999451


In [102]:
trips['is_round_trip'] = trips['start_station'] == trips['end_station']
round_trip_count_by_station = trips.groupby('start_station').agg(round_trip_count=('is_round_trip', 'sum'))
top_round_trip_stations = round_trip_count_by_station.nlargest(5, 'round_trip_count')
print("Top 5 stations most frequently used for round trips:")
print(top_round_trip_stations)

Top 5 stations most frequently used for round trips:
               round_trip_count
start_station                  
58                         3064
36                         2548
42                         2163
53                         2144
52                         1636


In [103]:
merged_trips = pd.merge(trips, stations, left_on='start_station', right_on='id', suffixes=('_start', '_end'))
merged_trips = pd.merge(merged_trips, stations, left_on='end_station', right_on='id', suffixes=('_start', '_end'))
different_municipalities_trips = merged_trips[merged_trips['municipality_start'] != merged_trips['municipality_end']]
trips_count = different_municipalities_trips['id_start'].count()

print("Number of trips that start and end in different municipalities:", trips_count)



Number of trips that start and end in different municipalities: 309748


In [104]:
trips['duration_minutes'] = trips['duration'] / 60
additional_fee_trips_count = trips[trips['duration_minutes'] > 30].shape[0]
print("Number of trips that lasted longer than 30 minutes (incurring additional fees):", additional_fee_trips_count)

Number of trips that lasted longer than 30 minutes (incurring additional fees): 123155


In [105]:
total_duration_by_bike = trips.groupby('bike_number')['duration'].sum()
most_used_bike = total_duration_by_bike.idxmax()
total_duration_longest_bike = total_duration_by_bike.max()
print("Bike number used for the longest total time:", most_used_bike)
print("Total duration for the longest used bike (in seconds):", total_duration_longest_bike)


Bike number used for the longest total time: B00490
Total duration for the longest used bike (in seconds): 2058643


In [74]:
round_trips_count_by_sub_type = trips[trips['start_station'] == trips['end_station']].groupby('sub_type').agg(round_trips_count=('start_station', 'size'))
print("Number of round trips for each sub_type:")
print(round_trips_count_by_sub_type)

Number of round trips for each sub_type:
            round_trips_count
sub_type                     
Casual                  41427
Registered              31635


In [106]:
merged_trips_stations = pd.merge(trips, stations, left_on='end_station', right_on='id')
station_end_count_by_municipality = merged_trips_stations.groupby('municipality').agg({'municipality': 'count'})
station_end_count_by_municipality = station_end_count_by_municipality.rename(columns={'municipality': 'station_end_count'})
most_frequent_station_end_municipality = station_end_count_by_municipality['station_end_count'].idxmax()
station_end_count_most_frequent_municipality = station_end_count_by_municipality['station_end_count'].max()

print("Municipality with the most frequent station ends:", most_frequent_station_end_municipality)
print("Number of station ends in the most frequent municipality:", station_end_count_most_frequent_municipality)

Municipality with the most frequent station ends: Boston
Number of station ends in the most frequent municipality: 1212364


In [107]:
trips_with_municipalities = trips.merge(stations, left_on='start_station', right_on='id', suffixes=('_start', '_end'))
trips_with_municipalities = trips_with_municipalities.merge(stations, left_on='end_station', right_on='id', suffixes=('_start', '_end'))
frequent_routes = trips_with_municipalities.groupby(['municipality_start', 'municipality_end']).size().nlargest(5).reset_index(name='trip_count')

print("Most frequent routes between municipalities:")
print(frequent_routes)

Most frequent routes between municipalities:
  municipality_start municipality_end  trip_count
0             Boston           Boston     1081805
1          Cambridge        Cambridge      162538
2             Boston        Cambridge      110968
3          Cambridge           Boston      110078
4          Cambridge       Somerville       20998


In [108]:
current_year = 2024
trips['age'] = current_year - trips['birth_date'].dt.year
trips['age_band'] = pd.cut(trips['age'], bins=[0, 10, 20, 30, 40, 50, 60, 70, float('inf')], labels=['1-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '70+'])
registered_trips = trips[trips['sub_type'] == 'Registered']
ride_count_by_age_band = registered_trips.groupby('age_band').size().reset_index(name='ride_count')
most_rides_age_band = ride_count_by_age_band.loc[ride_count_by_age_band['ride_count'].idxmax()]
print("Age band with the most rides for registered users:")
print(most_rides_age_band)


Age band with the most rides for registered users:
age_band       51-60
ride_count    350016
Name: 5, dtype: object


### BONUS PYTHON QUESTION (10 Marks)

##### Can you show a Report in Python "What are the Top 5 States with Most Number of Rides" For Registered Users Only

##### Hints :

1. Use the Sampled Dataframe for your working, else it would take a lot of time to convert many rows in Original table.

2. You will need Data Type Conversion and Cleaning of Zip Code (Don't fear errors / Learn from them)

3. Where do you get the States? READ THE CELL BELOW

!pip install zipcodes

print('\n')

import zipcodes

##### Gets All the Information Available against ZipCode

print(zipcodes.matching('77429')[0].keys())

['zip_code', 'zip_code_type', 'active', 'city', 'acceptable_cities', 'unacceptable_cities', 'state', 'county', 'timezone', 'area_codes', 'world_region', 'country', 'lat', 'long']

##### Sample Working how to Fetch a Against a ZipCode

zipcodes.matching('77429')[0].get('state')

READ MORE > https://github.com/seanpianka/Zipcodes


In [78]:

!pip install zipcodes

print('\n')

import zipcodes

Defaulting to user installation because normal site-packages is not writeable




In [79]:
#EXECUTE THIS CELL TO CREATE SAMPLED DATAFRAME

registered_trips_sampled=trips.query("sub_type=='Registered'").groupby('zip_code').apply(lambda x: x.sample(10, replace=True)).reset_index(drop = True)

In [80]:
import pandas as pd
import zipcodes

In [109]:
def get_state(zip_code):
    if pd.isnull(zip_code):
        zip_code = '0'
    
    zip_code = ''.join(filter(str.isdigit, str(zip_code)))
    try:
        state = zipcodes.matching(zip_code)[0].get('state')
    except:
        state = None
    return state

registered_trips_sampled['state'] = registered_trips_sampled['zip_code'].apply(get_state)

In [110]:
top_states = registered_trips_sampled['state'].value_counts().nlargest(5)

print("Top 5 States with the Most Number of Rides for Registered Users:")
print(top_states)

Top 5 States with the Most Number of Rides for Registered Users:
state
MA    3040
NH     460
NY     200
RI     190
CT     160
Name: count, dtype: int64


In [83]:
# WE WILL USE THIS DICTIONARY TO MAP STATE NAMES INSTEAD OF ABBREVIATIONS

state_names = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}


In [111]:
def get_state(zip_code):
    if pd.isnull(zip_code):
        zip_code = '0'
    zip_code = ''.join(filter(str.isdigit, str(zip_code)))
    try:
        state_abbr = zipcodes.matching(zip_code)[0].get('state')
        state_name = state_names.get(state_abbr)
    except:
        state_name = None
    return state_name

registered_trips_sampled['state'] = registered_trips_sampled['zip_code'].apply(get_state)

top_states = registered_trips_sampled['state'].value_counts().nlargest(5)

print("Top 5 States with the Most Number of Rides for Registered Users:")
print(top_states)


Top 5 States with the Most Number of Rides for Registered Users:
state
Massachusetts    3040
New Hampshire     460
New York          200
Rhode Island      190
Connecticut       160
Name: count, dtype: int64


### WE HOPE YOU ENJOYED LEARNING SQL & PYTHON TOGHETHER